Zachary Strasser

Import necessary modules

In [ ]:
import pandas as pd
import numpy as np
import itertools
import math as m
from matplotlib import pyplot as plt

Read tsv file

In [ ]:
tsv_file= '/content/sample_data/BindingDB_PDSPKi3.tsv'
tsv_file = pd.read_table(tsv_file, sep='\t', error_bad_lines=False)

b'Skipping line 1717: expected 49 fields, saw 85\nSkipping line 1718: expected 49 fields, saw 85\nSkipping line 1719: expected 49 fields, saw 85\nSkipping line 1720: expected 49 fields, saw 85\nSkipping line 1721: expected 49 fields, saw 85\nSkipping line 3561: expected 49 fields, saw 85\nSkipping line 3562: expected 49 fields, saw 85\nSkipping line 3563: expected 49 fields, saw 85\nSkipping line 3564: expected 49 fields, saw 85\nSkipping line 3565: expected 49 fields, saw 85\nSkipping line 3566: expected 49 fields, saw 85\nSkipping line 3567: expected 49 fields, saw 85\nSkipping line 3568: expected 49 fields, saw 85\nSkipping line 3569: expected 49 fields, saw 85\nSkipping line 3570: expected 49 fields, saw 85\nSkipping line 3571: expected 49 fields, saw 85\nSkipping line 3572: expected 49 fields, saw 85\nSkipping line 3573: expected 49 fields, saw 85\nSkipping line 3574: expected 49 fields, saw 85\nSkipping line 4976: expected 49 fields, saw 85\n'


Convert it to CSV file and save

In [ ]:
#tsv_file.to_csv('/content/sample_data/BindingFile.csv')

Check columns

In [ ]:
tsv_file.columns

Index(['BindingDB Reactant_set_id', 'Ligand SMILES', 'Ligand InChI',
       'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Name Assigned by Curator or DataSource',
       'Target Source Organism According to Curator or DataSource', 'Ki (nM)',
       'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)', 'kon (M-1-s-1)', 'koff (s-1)',
       'pH', 'Temp (C)', 'Curation/DataSource', 'Article DOI', 'PMID',
       'PubChem AID', 'Patent Number', 'Authors', 'Institution',
       'Link to Ligand in BindingDB', 'Link to Target in BindingDB',
       'Link to Ligand-Target Pair in BindingDB', 'Ligand HET ID in PDB',
       'PDB ID(s) for Ligand-Target Complex', 'PubChem CID', 'PubChem SID',
       'ChEBI ID of Ligand', 'ChEMBL ID of Ligand', 'DrugBank ID of Ligand',
       'IUPHAR_GRAC ID of Ligand', 'KEGG ID of Ligand', 'ZINC ID of Ligand',
       'Number of Protein Chains in Target (>1 implies a multichain complex)',
       'BindingDB Target Chain  Sequence', 'PDB ID(s) of T

Pull the necessary columns

In [ ]:
tsv_file_short = tsv_file[['Ligand SMILES', 'BindingDB Target Chain  Sequence', 'Ki (nM)']]

27,711 SMILE and protein sequence pairs with associated Ki values. We will split the group into 70/30 for training and validating.

In [ ]:
tsv_file_short

,Ligand SMILES,BindingDB Target Chain Sequence,Ki (nM)
0,Cn1c2ncn(CCN3CCC(CC3)C(=O)c3ccc(F)cc3)c2c(=O)n...,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,4.5
1,NC(N)=NN=Cc1c(Cl)cccc1Cl,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,199.5
2,Fc1ccc(cc1)C(=O)C1CCN(CCn2c(=O)[nH]c3ccccc3c2=...,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,3.2
3,Cc1nc2ccccn2c(=O)c1CCN1CCC(CC1)=C(c1ccc(F)cc1)...,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,6
4,Cc1nc2sccn2c(=O)c1CCN1CCC(CC1)=C(c1ccc(F)cc1)c...,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,5.5
...,...,...,...
7807,NCCc1c[nH]c2ccc(cc12)C(N)=O,MMDVNSSGRPDLYGHLRSLILPEVGRGLQDLSPDGGAHPVVSSWMP...,0.83
7808,NCCc1c[nH]c2ccc(cc12)C(N)=O,MMDVNSSGRPDLYGHLRSFLLPEVGRGLPDLSPDGGADPVAGSWAP...,0.93
7809,NCCc1c[nH]c2ccc(cc12)C(N)=O,MMDVNSSGRPDLYGHLRSLILPEVGRRLQDLSPDGGAHSVVSSWMP...,1
7810,NCCc1c[nH]c2ccc(cc12)C(N)=O,MMDVNSSGRPDLYGHLRSLILPEVGRGLQDLSPDGGAHPVVSSWMP...,0.15


Check to see if an rows have NaN

In [ ]:
tsv_file_short[['Ligand SMILES']].isnull().values.any()

False

No rows have NaN in BindingDB

In [ ]:
tsv_file_short[['BindingDB Target Chain  Sequence']].isnull().values.any()

True

Some rows have NaN in BindingDB target Chain, remove those rows. Find the row

In [ ]:
tsv_file_short[['BindingDB Target Chain  Sequence']].isnull().sum()

BindingDB Target Chain  Sequence    1
dtype: int64

It is the only final row with a NaN. Delete that row. 

In [ ]:
tsv_file_short = tsv_file_short.drop(7811)

Check final column for null values. None found

In [ ]:
tsv_file_short[['Ki (nM)']].isnull().values.any()

False

Convert PANDA into np.array

In [ ]:
DBBind = tsv_file_short.to_numpy()

Check the length of each of the SMILES. Starting with the minimum and maximum

In [ ]:
min([len(x) for x in DBBind[:,0].tolist()])

8

Minimum SMILE is a length of 8

In [ ]:
max([len(x) for x in DBBind[:,0].tolist()])

764

Maximum SMILE is a length of 764

Now check minimum and maximum of the protein

In [ ]:
min([len(x) for x in DBBind[:,1].tolist()])

11

Minimum protein AA is 11

In [ ]:
max([len(x) for x in DBBind[:,1].tolist()])

2353

Maximum protein AA is 2353

The vast majority of the ligands fall between 20 and 75 length. Therefore we removed any combinations with a SMILE length greater than 90.

In [ ]:
value = len(DBBind[:,0])
place_holder = []
for x in range((value)):
  if len(DBBind[x,0]) > 90:
    place_holder.append(x)

In [ ]:
DBBind = np.delete(DBBind, place_holder, axis=0)

Now we remove all proteins greater than 990 AA, which is about 100 pairs

In [ ]:
value = len(DBBind[:,0])
place_holder = []
for x in range((value)):
  if len(DBBind[x,1]) > 990:
    place_holder.append(x)

In [ ]:
DBBind = np.delete(DBBind, place_holder, axis=0)

Our new shape is (6,826 by 3) representing 6826 pairs

In [ ]:
DBBind.shape

(6826, 3)

For now we added 0s to get the ligand sizes to all equal 800

We then add on 0s to every protein AA sequence to get it to 2400 AA's.

In [ ]:
for x in range(len(DBBind[:,0])):
  DBBind[x,0] = DBBind[x,0][::-1]
  DBBind[x,0] = DBBind[x,0].zfill(100)       #fill ligand to 100
  DBBind[x,0] = DBBind[x,0][::-1]
  DBBind[x,1] = DBBind[x,1][::-1]
  DBBind[x,1] = DBBind[x,1].zfill(1000)     #fill protein to 2400
  DBBind[x,1] = DBBind[x,1][::-1]  
  DBBind[x,2] = (DBBind[x,2]).strip()        #strip sides
  if '>' == DBBind[x,2][0] :            #if Ki >10000 treat as 10000, Ki >70000 treat as 10000, ect.
    DBBind[x,2] = DBBind[x,2][1:]
  DBBind[x,2] = float(DBBind[x,2])        #convert Ki to int

In [ ]:
DBBind

array([['Cn1c2ncn(CCN3CCC(CC3)C(=O)c3ccc(F)cc3)c2c(=O)n(C)c1=O00000000000000000000000000000000000000000000000',
        'MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWTIDAENRTNLSCEGYLPPTCLSILHLQEKNWSALLTTVVIILTIAGNILVIMAVSLEKKLQNATNYFLMSLAIADMLLGFLVMPVSMLTILYGYRWPLPSKLCAIWIYLDVLFSTASIMHLCAISLDRYVAIQNPIHHSRFNSRTKAFLKIIAVWTISVGISMPIPVFGLQDDSKVFKEGSCLLADDNFVLIGSFVAFFIPLTIMVITYFLTIKSLQKEATLCVSDLSTRAKLASFSFLPQSSLSSEKLFQRSIHREPGSYAGRRTMQSISNEQKACKVLGIVFFLFVVMWCPFFITNIMAVICKESCNENVIGALLNVFVWIGYLSSAVNPLVYTLFNKTYRSAFSRYIQCQYKENRKPLQLILVNTIPALAYKSSQLQVGQKKNSQEDAEQTVDDCSMVTLGKQQSEENCTDNIETVNEKVSCV000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Curated Dataframe

In [ ]:
curated_dataframe = pd.DataFrame(data=DBBind)

In [ ]:
curated_dataframe.columns = ['SMILES', "Protein", "Ki"]

In [ ]:
curated_dataframe.to_excel("curated_df.xlsx")

For testing, we will compare zinc-15 database SMILes to acid sequence of 6M2Q SARS-CoV-2 3CL protease. 306 AA:

SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ


Here is the imported zinc database

In [ ]:
tsv_file= '/content/sample_data/in-cells.csv'
zinc_15 = pd.read_csv(tsv_file)

In [ ]:
zinc_15

,zinc_id,smiles
0,ZINC000245189325,O=P(=O)O
1,ZINC000029747110,COc1cccc2c1[C@@H]1CN(CCCCn3c(O)nc4c(sc5ncc(-c6...
2,ZINC000137550338,CN(C)c1cc(CNCC(C)(C)C)c(O)c2c1C[C@H]1C[C@H]3[C...
3,ZINC000137550409,CN(C)c1cc(CNCC(C)(C)C)c(O)c2c1C[C@H]1C[C@H]3[C...
4,ZINC000137550489,CN(C)c1cc(CNCC(C)(C)C)c(O)c2c1C[C@H]1C[C@H]3[C...
...,...,...
129560,ZINC000001536786,O=C1CC2(CCCC2)CC(=O)N1CCNC[C@H]1COc2ccccc2O1
129561,ZINC000001541570,CCCCCCCCCOc1ccc2[nH]cc(CCN)c2c1
129562,ZINC000001069090,COc1cc2c(cc1OC)[C@@]13CCN4CC5=CCO[C@H]6CC(=O)N...
129563,ZINC000002000919,COc1ccc([C@H]2CNC(=O)C2)cc1OC1CCCC1


The following sequence is the 3CL protease

In [ ]:
"SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ"

In [ ]:
import torch